<a href="https://colab.research.google.com/github/anishjadavpur/Chromium-detector-API/blob/main/Chromium_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from joblib import load
import numpy as np
import pandas as pd

In [47]:
df= pd.read_csv(r"/content/Chromium_cleaned_final.csv")
df.set_index("TestNO",inplace=True)

In [48]:
df

,B,G,R,b,H,Conc
TestNO,,,,,,
1,78.79,119.07,56.63,144.2,70.60,500
1,88.77,120.10,80.05,140.8,66.58,250
2,92.00,129.00,94.00,144.0,58.00,25
2,92.00,129.00,92.00,144.0,60.00,50
2,89.54,122.94,85.67,142.2,63.12,250
...,...,...,...,...,...,...
496,94.00,131.00,96.00,144.0,58.00,25
497,92.00,129.00,94.00,144.0,58.00,25
498,93.00,130.00,96.00,144.0,58.00,25


In [49]:
# df=df.drop(columns=['L','a','S','I'])

In [50]:
# File names
filename1 = 'best_chromium_regressor_pipeline_Model1.joblib'
filename2 = 'best_chromium_regressor_pipeline2 (2).joblib'
filename3 = 'best_chromium_regressor_pipeline3.joblib'


In [51]:
import joblib
import time
import numpy as np
import psutil
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

In [52]:
def calculate_adjusted_r2(r2, n, p):
    if n > p:
        adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    else:
        adj_r2 = float('-inf')  # Or some other indicator of invalid result
    return adj_r2



def measure_resources(filename, X_test, y_test):
    # Load the model
    model = joblib.load(filename)

    # Record initial resource usage
    process = psutil.Process(os.getpid())
    initial_cpu = psutil.cpu_percent(interval=None)
    initial_memory = process.memory_info().rss

    # Measure prediction time
    start_time = time.time()
    y_pred = model.predict(X_test)
    prediction_time = time.time() - start_time

    # Record resource usage after prediction
    final_cpu = psutil.cpu_percent(interval=None)
    final_memory = process.memory_info().rss

    # Calculate resource usage
    cpu_usage = final_cpu - initial_cpu
    memory_usage = (final_memory - initial_memory) / (1024 * 1024)  # Convert to MB

    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    adj_r2 = calculate_adjusted_r2(r2, len(y_test), X_test.shape[1])

    # Print the results
    print(f"Prediction Time for model loaded from {filename}: {prediction_time} seconds")
    print(f"CPU Usage: {cpu_usage}%")
    print(f"Memory Usage: {memory_usage:.2f} MB")
    print("Performance Metrics:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"R-squared: {r2}")
    print(f"Adjusted R-squared: {adj_r2}")

    return prediction_time, cpu_usage, memory_usage, mse, rmse, mae, mape, r2, adj_r2



In [53]:
random_states = [21, 42, 100, 200]

# def get_random_test_data(df, num_samples=6, num_iterations=10, random_state=21):
#     np.random.seed(random_state)  # Set the random seed for reproducibility
#     X_tests = []
#     y_tests = []
#     for _ in range(num_iterations):
#         sampled_df = df.sample(n=num_samples, random_state=random_state)
#         X_test = sampled_df.drop(columns=['Conc']).values
#         y_test = sampled_df['Conc'].values
#         X_tests.append(X_test)
#         y_tests.append(y_test)
#     return X_tests, y_tests

# # Get random test data
# X_tests, y_tests = get_random_test_data(df, num_samples=20, num_iterations=5)

def get_random_test_data(df, num_samples=20, random_state=21):
    np.random.seed(random_state)  # Set the random seed for reproducibility
    sampled_df = df.sample(n=num_samples, random_state=random_state)
    X_test = sampled_df.drop(columns=['Conc']).values
    y_test = sampled_df['Conc'].values
    return X_test, y_test

def evaluate_for_random_states(df, random_states, num_samples=30):
    results = {state: [] for state in random_states}

    for state in random_states:
        X_tests = []
        y_tests = []
        for _ in range(10):  # Number of iterations
            X_test, y_test = get_random_test_data(df, num_samples=num_samples, random_state=state)
            X_tests.append(X_test)
            y_tests.append(y_test)

        for X_test, y_test in zip(X_tests, y_tests):
            print(f"Evaluating Model 1 - Random State {state}")
            result1 = measure_resources(filename1, X_test, y_test)
            print(f"Evaluating Model 2 - Random State {state}")
            result2 = measure_resources(filename2, X_test, y_test)
            print(f"Evaluating Model 3 - Random State {state}")
            result3 = measure_resources(filename3, X_test, y_test)

            results[state].append((result1, result2, result3))

    return results

def average_results(results):
    num_states = len(results)

    all_results1 = []
    all_results2 = []
    all_results3 = []

    for state in results:
        for result1, result2, result3 in results[state]:
            all_results1.append(result1)
            all_results2.append(result2)
            all_results3.append(result3)

    def average(metrics):
        return np.mean(metrics, axis=0)

    avg_results1 = average(all_results1)
    avg_results2 = average(all_results2)
    avg_results3 = average(all_results3)

    return avg_results1, avg_results2, avg_results3

# Example usage
results = evaluate_for_random_states(df, random_states=[21, 42, 100])
average_results1, average_results2, average_results3 = average_results(results)

print("Average Results for Model 1:", average_results1)
print("Average Results for Model 2:", average_results2)
print("Average Results for Model 3:", average_results3)


Evaluating Model 1 - Random State 21
Prediction Time for model loaded from best_chromium_regressor_pipeline_Model1.joblib: 0.018854618072509766 seconds
CPU Usage: 46.800000000000004%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 33.607929268299266
RMSE: 5.79723462249884
MAE: 1.8589353561401367
MAPE: 0.019510205586751303
R-squared: 0.9990260335658526
Adjusted R-squared: 0.998823123892072
Evaluating Model 2 - Random State 21


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Prediction Time for model loaded from best_chromium_regressor_pipeline2 (2).joblib: 0.039022207260131836 seconds
CPU Usage: 8.900000000000006%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 21.551682045663984
RMSE: 4.64237892094818
MAE: 1.050514030456543
MAPE: 0.0067021443684895845
R-squared: 0.9993754267112287
Adjusted R-squared: 0.999245307276068
Evaluating Model 3 - Random State 21


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Prediction Time for model loaded from best_chromium_regressor_pipeline3.joblib: 0.051416635513305664 seconds
CPU Usage: 6.799999999999997%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 888.1082513881265
RMSE: 29.80114513551663
MAE: 5.985460662841797
MAPE: 0.03223276977539063
R-squared: 0.9742623944535229
Adjusted R-squared: 0.9689003932980068
Evaluating Model 1 - Random State 21
Prediction Time for model loaded from best_chromium_regressor_pipeline_Model1.joblib: 0.01756429672241211 seconds
CPU Usage: 6.599999999999994%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 33.607929268299266
RMSE: 5.79723462249884
MAE: 1.8589353561401367
MAPE: 0.019510205586751303
R-squared: 0.9990260335658526
Adjusted R-squared: 0.998823123892072
Evaluating Model 2 - Random State 21


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Prediction Time for model loaded from best_chromium_regressor_pipeline3.joblib: 0.05105924606323242 seconds
CPU Usage: 8.299999999999997%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 8.49481618643913
RMSE: 2.914586795145948
MAE: 1.2942095438639323
MAPE: 0.02393953959147136
R-squared: 0.999680987474026
Adjusted R-squared: 0.9996145265311147
Evaluating Model 1 - Random State 100
Prediction Time for model loaded from best_chromium_regressor_pipeline_Model1.joblib: 0.01666712760925293 seconds
CPU Usage: 15.0%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 15.416265185907347
RMSE: 3.9263552037363287
MAE: 1.6170894622802734
MAPE: 0.040411225891113296
R-squared: 0.9994210608458024
Adjusted R-squared: 0.9993004485220113
Evaluating Model 2 - Random State 100


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Prediction Time for model loaded from best_chromium_regressor_pipeline2 (2).joblib: 0.029939889907836914 seconds
CPU Usage: -11.400000000000006%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 8.768265544008196
RMSE: 2.961125722425206
MAE: 0.9930341720581055
MAPE: 0.02219803110758464
R-squared: 0.9996707184148292
Adjusted R-squared: 0.9996021180845853
Evaluating Model 3 - Random State 100
Prediction Time for model loaded from best_chromium_regressor_pipeline3.joblib: 0.048384666442871094 seconds
CPU Usage: 3.4000000000000057%
Memory Usage: 0.00 MB
Performance Metrics:
MSE: 8.49481618643913
RMSE: 2.914586795145948
MAE: 1.2942095438639323
MAPE: 0.02393953959147136
R-squared: 0.999680987474026
Adjusted R-squared: 0.9996145265311147
Average Results for Model 1: [2.23713557e-02 3.12000000e+00 0.00000000e+00 2.37905324e+01
 4.81696459e+00 1.71505330e+00 3.15854172e-02 9.99277562e-01
 9.99127054e-01]
Average Results for Model 2: [ 0.04360418  2.60666667  0.         24.98327473  4.76135475  1.

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [54]:

print("Average Results for Model 1:")
print(f"Prediction Time: {average_results1[0]} seconds")
print(f"CPU Usage: {average_results1[1]}%")
print(f"Memory Usage: {average_results1[2]:.2f} MB")
print(f"MSE: {average_results1[3]}")
print(f"RMSE: {average_results1[4]}")
print(f"MAE: {average_results1[5]}")
print(f"MAPE: {average_results1[6]}")
print(f"R-squared: {average_results1[7]}")
print(f"Adjusted R-squared: {average_results1[8]}")

print("\nAverage Results for Model 2:")
print(f"Prediction Time: {average_results2[0]} seconds")
print(f"CPU Usage: {average_results2[1]}%")
print(f"Memory Usage: {average_results2[2]:.2f} MB")
print(f"MSE: {average_results2[3]}")
print(f"RMSE: {average_results2[4]}")
print(f"MAE: {average_results2[5]}")
print(f"MAPE: {average_results2[6]}")
print(f"R-squared: {average_results2[7]}")
print(f"Adjusted R-squared: {average_results2[8]}")

print("\nAverage Results for Model 3:")
print(f"Prediction Time: {average_results3[0]} seconds")
print(f"CPU Usage: {average_results3[1]}%")
print(f"Memory Usage: {average_results3[2]:.2f} MB")
print(f"MSE: {average_results3[3]}")
print(f"RMSE: {average_results3[4]}")
print(f"MAE: {average_results3[5]}")
print(f"MAPE: {average_results3[6]}")
print(f"R-squared: {average_results3[7]}")
print(f"Adjusted R-squared: {average_results3[8]}")

Average Results for Model 1:
Prediction Time: 0.022371355692545572 seconds
CPU Usage: 3.12%
Memory Usage: 0.00 MB
MSE: 23.790532355886544
RMSE: 4.816964591294556
MAE: 1.7150533040364593
MAPE: 0.031585417175292986
R-squared: 0.9992775619694859
Adjusted R-squared: 0.9991270540464626

Average Results for Model 2:
Prediction Time: 0.04360417524973551 seconds
CPU Usage: 2.6066666666666687%
Memory Usage: 0.00 MB
MSE: 24.983274726547293
RMSE: 4.761354748310486
MAE: 1.3531895743476017
MAPE: 0.02844143185085722
R-squared: 0.9992730377452325
Adjusted R-squared: 0.9991215872754888

Average Results for Model 3:
Prediction Time: 0.06407434145609538 seconds
CPU Usage: 8.000000000000002%
Memory Usage: 0.00 MB
MSE: 313.05934030588713
RMSE: 13.080226820832506
MAE: 3.230888133578828
MAPE: 0.03622601411607532
R-squared: 0.9909242823622983
Adjusted R-squared: 0.9890335078544442


In [55]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
import joblib
import scipy

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("XGBoost version:", xgb.__version__)
print("scikit-learn version:", sklearn.__version__)
print("joblib version:", joblib.__version__)
print("SciPy version:", scipy.__version__)


NumPy version: 1.26.4
Pandas version: 2.1.4
XGBoost version: 2.1.0
scikit-learn version: 1.3.2
joblib version: 1.4.2
SciPy version: 1.13.1
